In [1]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [3]:
doc_reader = PyPDFLoader("The_3_Alarms.pdf")
doc_reader

In [4]:
pages = doc_reader.load_and_split()
pages[0]

Document(page_content='THE 3 ALARMSA Simple System to Transform Your Health, Wealth, and Relationships ForeverERIC PARTAKER', metadata={'source': 'The_3_Alarms.pdf', 'page': 1})

In [5]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [6]:
texts = text_splitter.create_documents([doc_reader])
print(texts[0])

TypeError: expected string or bytes-like object, got 'PyPDFLoader'